### Include those who realized turnover

drop the rest 

In [92]:
import pandas as pd
import numpy as np
import seaborn as sns
import math

import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.formula.api import ols



# define path: insert the path where the SOEP data is stored on your computer here
from pathlib import Path
## copy your path in here:
# Path Max: C:/Users/max-admin/Desktop/Masterstudium/WiSe_22_23/Research_Module/SOEP-Data/Stata/raw
# Path Maxie: /Volumes/dohmen_soep/SOEP-CORE.v36eu_STATA/Stata/raw

data_folder = Path("/Volumes/dohmen_soep/SOEP-CORE.v36eu_STATA/Stata/raw")
# define relevant subsets of SOEP-data
file_names = ['vp', 'wp', 'xp']

file_paths = [data_folder / f"{file_name}.dta" for file_name in file_names]
# some controls are in gen data
file_paths_2 = [data_folder / f"{file_name}gen.dta" for file_name in file_names]


In [93]:
# read in 2005 data for the reciprocity measures
data05 = pd.read_stata(file_paths[0], columns=["pid","hid", "syear","vp12602", "vp12603", "vp12605"]).set_index(['pid', 'hid'])
df_2005 = data05.rename(columns={ 'vp12602': 'take_revenge', 'vp12603': 'similar_problems', 'vp12605': 'insult_back'})
# create dummies for take_revenge question
# Create dummy variables
dummies = pd.get_dummies(df_2005['take_revenge'])

# Join the dummy variables to the original dataframe
df_2005 = pd.concat([df_2005, dummies], axis=1)
# rename dummy
df_05 = df_2005.rename(columns = {'[1] Trifft ueberhaupt nicht zu' : 'rec1' , '[2] Skala 1-7' : 'rec2' , '[3] Skala 1-7' : 'rec3' ,'[4] Skala 1-7' : 'rec4' ,'[5] Skala 1-7' : 'rec5' ,'[6] Skala 1-7' : 'rec6' ,'[7] Trifft voll zu' : 'rec7'})
df_05 = df_05.drop(columns = ["[-1] keine Angabe"])
df_05.head(3)

,,syear,take_revenge,similar_problems,insult_back,rec1,rec2,rec3,rec4,rec5,rec6,rec7
pid,hid,,,,,,,,,,,
201,27,2005,[1] Trifft ueberhaupt nicht zu,[5] Skala 1-7,[1] Trifft ueberhaupt nicht zu,1,0,0,0,0,0,0
203,60313,2005,[2] Skala 1-7,[3] Skala 1-7,[2] Skala 1-7,0,1,0,0,0,0,0
602,60,2005,[5] Skala 1-7,[4] Skala 1-7,[3] Skala 1-7,0,0,0,0,1,0,0


In [90]:
# read in 2006 data
# personal = personal advancement
# still includes all unfair treat
data06 = pd.read_stata(file_paths[1], columns=["pid", "hid", "syear", 'wp43b01', 'wp43b02', 'wp43b03', 'wp43b04', 'wp43b05', 'wp43b06', 'wp43b07','wp43b08' ]).set_index(['pid', 'hid'])
df_06 = data06.rename(columns={ 'wp43b01': 'recog_sup', 'wp43b02': 'felt_recog_sup',"wp43b03": "recog_effort",  'wp43b04': 'felt_recog_effort', "wp43b05": "recog_personal", "wp43b06" :"felt_recog_personal" ,"wp43b07": "recog_pay",'wp43b08': 'felt_recog_pay'})


## this next one one has to include ALL of the following columns: done
data2 = pd.read_stata(file_paths[1], columns=["pid", "hid", "syear","wp12402", "wp12401", "wp5802",  "wp5902", "wp4101", "wp17", "wp1901"  ]).set_index(['pid', 'hid'])
df_06insteadof07 = data2.rename(columns={"wp12401": "gender", "wp12402": "year_birth", "wp5902": "wage_lastmonth", "wp4101": "commute_distance", 'wp5802': 'overtime', 'wp17': 'new_job', 'wp1901': 'reason_new_job'})
# add turnover intention = 1. dont add turnover intention above
df_06insteadof07["turnover_intention"] = '[100] 100% wahrscheinlich'
# change has to include all of the following
hours06 = pd.read_stata(file_paths_2[1], columns=["pid","hid", "syear", "wvebzeit", "betr06", "wpsbil", "werwzeit", "wbilzeit"]).set_index(['pid', 'hid'])
df_work06insteadof07 = hours06.rename(columns={'wvebzeit': 'working_hours', "betr06": "firmsize", "wpsbil": "school_degree", "werwzeit": "tenure" , "wbilzeit" : "years_educ"})

Mappings und Functionen

Hier könnte es sein dass die Mappings nicht die selben sind in beiden Jahren. bitte checken (am ende wenn zu viele n.a.ns oder so sind)

In [91]:
# mapping for reciprocity questions: same scale for all
reciprocity_questions_mapping = {
    '[1] Trifft ueberhaupt nicht zu': 1,
    '[2] Skala 1-7': 2,
    '[3] Skala 1-7': 3,
    '[4] Skala 1-7': 4,
    '[5] Skala 1-7': 5,
    '[6] Skala 1-7': 6,
    '[7] Trifft voll zu': 7,
    '[-1] keine Angabe': -1,
}
## mapping for recognition questions: binary -> binary -> unfair treatment: No -> later 1 fair treatment: Yes -> 0
recog_mapping = {
    '[-2] trifft nicht zu': -2,
    '[-1] keine Angabe': -1,
    '[1] Ja': 2,
    '[2] Nein': 1,
}
# felt recog mapping
felt_recog_mapping = {
    '[-2] trifft nicht zu': -2,
    '[-1] keine Angabe': -1,
    '[1] Gar nicht': 1,
    '[2] Maessig': 2,
    '[3] Stark': 3,
    '[4] Sehr stark': 4,
}
# mapping for firmsize -> we need to recode this in a sensible way: jumps are the same: first change: selbstständig to 0
firmsize_mapping = {
    '[-2] trifft nicht zu': -2,
    '[-1] keine Angabe': -1,
    '[1] Unter  5': 1,
    '[2] 5 bis 10': 2,
    '[3] 11 bis unter 20': 3,
    '[4] bis 90: unter 20': 4,
    '[5] 91-04: 5 bis unter 20': 5,
    '[6] 20 bis unter 100': 6,
    '[7] 100 bis unter 200': 7,
    '[8] bis 98: 20 bis unter 200': 8,
    '[9] 200 bis unter 2000': 9,
    '[10] 2000 und mehr': 10,
    '[11] Selbstaendig-ohne Mitarb.': 0,
}
# mapping new job into binary variable
new_job_mapping = {
    '[-2] trifft nicht zu': -2,
    '[-1] keine Angabe': -1, 
    '[1] Ja': 1, 
    '[2] Nein': 2,
    '[3] Ja, nach Datenpruefung': 1,
}
# mapping for turnover intention: split up into binary with roughly equal value counts for simplicity: might change that later to categories
turnover_mapping = {
    '[-2] trifft nicht zu': -2,
    '[-1] keine Angabe':-1,
    '[0] 0% wahrscheinlich': 2,
    '[10] 10% wahrscheinlich': 1,
    '[20] 20% wahrscheinlich': 1,
    '[30] 30% wahrscheinlich': 1,
    '[40] 40% wahrscheinlich': 1, 
    '[50] 50% wahrscheinlich': 1,
    '[60] 60% wahrscheinlich': 1,
    '[70] 70% wahrscheinlich': 1,
    '[80] 80% wahrscheinlich': 1,
    '[90] 90% wahrscheinlich': 1,
    '[100] 100% wahrscheinlich': 1,
}
# mapping for turnover intention ROBUSTNESS CHECK: Cardinal
turnover_mapping_cardinal = {
    '[-2] trifft nicht zu': -2,
    '[-1] keine Angabe':-1,
    '[0] 0% wahrscheinlich': 0,
    '[10] 10% wahrscheinlich': 10,
    '[20] 20% wahrscheinlich': 20,
    '[30] 30% wahrscheinlich': 30,
    '[40] 40% wahrscheinlich': 40, 
    '[50] 50% wahrscheinlich': 50,
    '[60] 60% wahrscheinlich': 60,
    '[70] 70% wahrscheinlich': 70,
    '[80] 80% wahrscheinlich': 80,
    '[90] 90% wahrscheinlich': 90,
    '[100] 100% wahrscheinlich': 100,
}
# mapping for new job to easier remove negatives
reason_new_job_mapping = {
    '[-2] trifft nicht zu': -2,
    '[-1] keine Angabe':-1, 
    '[1] Erstmals erwerbstaetig': 1,
    '[2] Wieder erwerbstaetig': 2,
    '[3] Stelle bei neuen Arbeitgeber': 3,  
    '[4] Uerbnommen von Betrieb': 4,
    '[5] Stellenwechsel im Betrieb': 5, 
    '[6] Selbstaendig geworden': 6,
}
# mapping for school degree: to easier remove negatives
school_degree_mapping = {
    '[-2] trifft nicht zu': -2,
    '[-1] keine Angabe':-1,
    '[1] Hauptschulabschluss': 1,
    '[2] Realschulabschluss': 2,
    '[3] Fachhochschulreife': 3,
    '[4] Abitur': 4,
    '[5] Anderer Abschluss': 5,
    '[6] Ohne Abschluss verlassen': 6,
    '[7] Noch kein Abschluss': 7,
    '[8] Keine Schule besucht': 8,
}
# reversed mapping to redo changes
reversed_mapping_reason = {v: k for k, v in reason_new_job_mapping.items()}
reversed_mapping_schoold = {v: k for k, v in school_degree_mapping.items()}
# mapping for binary reciprocity measure: ROBUSTNESS CHECK
def binary_reciprocity(x):
    if x >= 3:
        return 1 # High Group
    else:
        return 0 # Low Group

def recode_categoricals(inputdf,rc_cardinal = 0):
    """
        recodes categoricals according to defined mappings

        Input:
            - inputdf : merged dataframe
            - rc_cardinal: optional argument: if == 1 turnover intentions will be coded as cardinal instead of binary 
            - rc_rec_binary: optional argument: if == 1 neg. reciprocity will be split into 2 groups: high and low
        Output:
            - df : cleaned Dataframe

    """
    merged = inputdf  
    
    # recode Gender variable
    merged['gender'].replace('[2] Weiblich', 2,inplace=True)
    merged['gender'].replace('[1] Maennlich', 1,inplace=True)
    # recode reciprocity variables
    merged[["similar_problems","take_revenge","insult_back"]] = merged[["similar_problems","take_revenge","insult_back"]].apply(lambda x: x.map(reciprocity_questions_mapping))
    # recode recognition variables
    merged[["recog_sup","recog_effort","recog_personal","recog_pay"]] = merged[["recog_sup","recog_effort","recog_personal","recog_pay"]].apply(lambda x: x.map(recog_mapping))
    # recode felt recognition variables
    merged[["felt_recog_sup","felt_recog_effort","felt_recog_personal","felt_recog_pay"]] = merged[["felt_recog_sup","felt_recog_effort","felt_recog_personal","felt_recog_pay"]].apply(lambda x: x.map(felt_recog_mapping))
    # recode firm size
    merged['firmsize'] = merged['firmsize'].map(firmsize_mapping)
    # recode new job reason variable
    merged['reason_new_job'] = merged['reason_new_job'].map(reason_new_job_mapping)
    # recode job change variable
    merged['new_job']= merged['new_job'].map(new_job_mapping)
    # recode turnover intention variable
    if rc_cardinal == 1:
        merged['turnover_intention'] = merged['turnover_intention'].map(turnover_mapping_cardinal)
    else:
        merged['turnover_intention'] = merged['turnover_intention'].map(turnover_mapping)
    # recode school degree
    merged['school_degree'] = merged['school_degree'].map(school_degree_mapping)
    
    output = merged
    return output

#def merge_and_clean(df_05,df_06,df_07,work07,rc_cardinal = 0,rc_rec_binary=0):
def merge_and_clean(df_05,df_06,df_07,work07,rc_cardinal = 0,rc_rec_binary=0):
    """
    merges data from different years, applies recoding to categoricals and constructs additional variables.

    Input:
        - df_05 : Pd.Dataframe contains reciprocity measures
        - df_06 : Pd.Dataframe contains unfair treatment measures
        - df_07 : Pd.Dataframe contains outcome and controls
        - work07 : Pd.Dataframe contains additional controls

    Output:
        - df : cleaned Dataframe

    """
    allmerged_df = pd.merge(pd.merge(pd.merge(df_05,df_06,on=["pid", "hid"]),df_07,on=["pid","hid"]),work07,on=["pid", "hid"])
    recoded = recode_categoricals(allmerged_df,rc_cardinal).astype(int)

    # replaces negative values with n.a.n 
    recoded = recoded.mask(recoded < 0, np.nan) 
    # construct avg reciprocity measure
    recoded['avg_rec'] = recoded[['take_revenge', 'similar_problems', 'insult_back']].mean(axis=1)
    # For robustness check: option to construct binary reciprocity measure.
    if rc_rec_binary == 1:
        recoded['binary_rec'] = recoded['avg_rec'].apply(binary_reciprocity)
    
    # construct age, potential experience and age^2
    recoded['age'] = 2007 - recoded['year_birth']
    recoded["potential_experience"] = pow((recoded["age"] - 18), 2)
    recoded["age_squared"] = (recoded["age"] ** 2) / 100
    # recode categoricals back to make it better readable
    recoded["reason_new_job"] = recoded["reason_new_job"].map(reversed_mapping_reason)
    recoded["school_degree"] = recoded["school_degree"].map(reversed_mapping_schoold)

    # transform binary variables with 1 and 2 into 1 and 0
    columns_to_transform = ["recog_sup","recog_effort", "recog_pay", "recog_personal" ,"gender", "turnover_intention", "new_job"]

    # Iterate over the columns and replace the values 2 with 0 
    for col in columns_to_transform:
        recoded[col] = recoded[col].replace({2: 0})

    # save df somewhere so its not muted when repeatedly executing this cell: Can later transform that into functions
    df = recoded
    
    return df

# Add Mincer Wage Regression and adds its residuals to the Dataframe

def add_mincer_residuals(cleaneddata):
    
    df_cleaned = cleaneddata
    # specify which columns to drop from our dataframe
    df_mincer = cleaneddata.drop(columns=['syear_x', 'similar_problems', 'take_revenge', 'insult_back','syear_y', 'felt_recog_sup', 'felt_recog_effort',
       'recog_personal', 'felt_recog_personal', 'recog_pay', 'felt_recog_pay', 'syear_y', 'year_birth', 'new_job',
       'reason_new_job', 'school_degree','overtime', 'recog_sup', 'age', 'commute_distance', 'recog_effort', 'working_hours', 'turnover_intention', 'rec1' , 'rec2' , 'rec3' , 'rec4', 'rec5' , 'rec6' , 'rec7'])
    # Convert 'gender' and 'sector' columns to categorical data type
    df_mincer['gender'] = df_mincer['gender'].astype('category')
    df_mincer = df_mincer.dropna()
    # Define the dependent variable
    y = df_mincer['wage_lastmonth']
    # Define the independent variables
    X = df_mincer[['gender', 'firmsize', 'tenure', 'years_educ', 'potential_experience', 'age_squared']]

    # Add a constant term to the independent variables
    X = sm.add_constant(X)

    # Fit the Mincer wage regression model
    mincer_model = sm.OLS(y, X).fit()
    # Create a new column in the dataframe with the same name as the residuals array/ delete relative wage entries
    df_cleaned['mincer_residuals'] = None
    # Match the rows of the dataframe with the values in the residuals array using the index
    df_cleaned.loc[df_cleaned.index, 'mincer_residuals'] = mincer_model.resid

    return df_cleaned

In [95]:
datafull06 = add_mincer_residuals(merge_and_clean(df_05,df_06,df_06insteadof07,df_work06insteadof07,rc_cardinal = 0,rc_rec_binary=0))
# save as xlsx
datafull06.to_csv('/Users/maxieschulze/Documents/Dokumente - MacBook Pro von Maxie/5. Semester/Research Module/ResearchModule/src/data_management/2006jobchange.csv', index=True)


/var/folders/7l/hw861dtx52z8nxgtzjvpp8_h0000gn/T/ipykernel_83865/3879379438.py:173: FutureWarning: Passing 'suffixes' which cause duplicate columns {'syear_x'} in the result is deprecated and will raise a MergeError in a future version.
  allmerged_df = pd.merge(pd.merge(pd.merge(df_05,df_06,on=["pid", "hid"]),df_07,on=["pid","hid"]),work07,on=["pid", "hid"])
/var/folders/7l/hw861dtx52z8nxgtzjvpp8_h0000gn/T/ipykernel_83865/3879379438.py:229: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df_cleaned.loc[df_cleaned.index, 'mincer_residuals'] = mincer_model.resid
